![title](../img/UNIBET.png)

In [11]:
# %run ./PProcess0.ipynb
# driver = webdriver.Chrome(ChromeDriverManager().install())

In [6]:
url = "https://www.unibet.fr/zones/v3/sportnode/markets.json?nodeId=2243762&filter=R%C3%A9sultat&marketname=R%C3%A9sultat%20du%20match"
response = requests.get(url)
json_data = response.json()

#Missing:
# (dfGames["competitionName"].str.contains("Nor")) |
# (dfGames["competitionName"].str.contains("Suède, Allsvenskan")) |

row_data1 = [
    {
        "Day": day,
        "Match": e["eventName"],
        "EventId": e["eventId"],
        "competitionName": e["competitionName"],
    }
    for day, d in enumerate(json_data["marketsByType"][0]["days"])
    for e in d["events"]
]

competitions = [    
#                 re.compile("Allemagne, Bundesliga\s*"),
#                 re.compile("Italie, Serie \s*"),
#                 re.compile("France, Ligue\s*"),
#                 re.compile("Espagne, LaLiga\s*"),
#                 re.compile("Angleterre, Championship\s*"),
#                 re.compile("Angleterre, Premier League\s*"),
#                 re.compile("Portugal, Primeira Liga\s*"),
#                 re.compile("Ecosse, Premiership\s*"),
#                 re.compile("Danemark, SuperLiga\s*"),
#                 re.compile("Autriche, Bundesliga\s*"),
#                 re.compile("Suisse, Super League\s*"),
#                 re.compile("Belgique, Pro League\s*"),
#                 re.compile("Pays-Bas, Eredivisie\s*"),
#                 re.compile("Nor\s*"),
#                 re.compile("Suède, Allsvenskan\s*"),
#                 re.compile("Turquie, Süper Lig\s*"),
#                 re.compile("Grèce, Super League\s*"),
#                 re.compile("Roumanie, Liga 1\s*"),
#                 re.compile("Israël, Premier League\s*"),
#                 re.compile("Irlande, Premier League\s*"),
#                 re.compile("Irlande du Nord, NIFL Premiership\s*"),
#                 re.compile("Mexique, Liga MX\s*"),
#                 re.compile("Etats-Unis, MLS\s*"),
#                 re.compile("Argentine, Liga Profesional\s*"),
                re.compile("Brésil\s*"),
#                 re.compile("Colombie, Primera A\s*"),
#                 re.compile("Chili, Primera División\s*"),
#                 re.compile("Equateur, Liga Pro\s*"),
#                 re.compile("Paraguay, Primera Division\s*"),
#                 re.compile("Japon, J-League\s*"),
#                 re.compile("Australie, A-League\s*"),
                ]

dfGames = pd.DataFrame(row_data1)
# dfGames = dfGames[(dfGames["Day"] >= -99) & (dfGames["competitionName"].isin(competitions))]
dfGames = dfGames[(dfGames["Day"] >= -99) & (dfGames["competitionName"].apply(lambda x: any(regex.match(x) for regex in competitions)))]

#dfGames
df = pd.DataFrame(dfGames['Match'].str.split(' - ', expand=True))
df.columns = ['Home', 'Away']
dfGames = pd.concat([df, dfGames], axis=1)
dfGames["Away"] = dfGames["Away"].apply(strip_accents)
dfGames["Home"] = dfGames["Home"].apply(strip_accents)
dfGames["Away"].replace(Dict_teams, inplace=True)
dfGames["Home"].replace(Dict_teams, inplace=True)
dfGames["Away"].replace(Match_teams, inplace=True)
dfGames["Home"].replace(Match_teams, inplace=True)
try:
    if (len(Check[mask]['IdGame'].unique())>0):
        if not goAllBets:
            dfGames = dfGames[dfGames["EventId"].isin(list(Check[mask]['IdGame'].unique()) + [dfGames["EventId"].unique()[0]])]
except:
    print("Dataset Check is not found")
    
dfGames['competitionName'].unique()

NameError: name 'driver' is not defined

In [46]:
# a = pd.DataFrame(row_data1)
# a = a['competitionName'].unique()
# b = dfGames['competitionName'].unique()
# list(set(a) ^ set(b))

In [20]:
#Get match odds
import datetime
row_data0 = []
nbGames = 0

for idgame in tqdm(dfGames["EventId"]):
    g = "https://www.unibet.fr/zones/event.json?eventId=" + idgame
#     with urllib.request.urlopen(g) as url:
#         json_data = json.loads(url.read().decode())
    response = requests.get(url)

    if response.status_code == 200:
        json_data = json.loads(response.text)
        # Now you can work with the JSON data
    else:
        print(f"Failed to retrieve data for EventId: {idgame}, Status Code: {response.status_code}")
        
        
    competitionName = json_data["eventHeader"]['competitionName']
    timestamp = datetime.datetime.fromtimestamp(json_data["eventHeader"]['eventStartDate']/1000)
    DateTime = timestamp.strftime('%Y-%m-%d %H:%M:%S')
    #home = json_data["eventHeader"]['homeName']
    #away = json_data["eventHeader"]['awayName']
    #i = -1
    if len(json_data['marketClassList'])>1:
        duel = json_data['marketClassList'][0]['marketList'][0]['selections']
#         if (" ou ") in json_data['marketClassList'][1]['marketList'][0]['selections'][1]['name']:
#             home = strip_accents(duel.split(" ou ")[0])
#             away = strip_accents(duel.split(" ou ")[1])
#         elif (" Or ") in json_data['marketClassList'][1]['marketList'][0]['selections'][1]['name']:
#             home = strip_accents(duel.split(" Or ")[0])
#             away = strip_accents(duel.split(" Or ")[1])
        home = strip_accents(duel[0]['name'])
        away = strip_accents(duel[2]['name'])

        for m in json_data['marketClassList']:
            #i = i + 1
            
            if(m['marketName'] == 'Total de buts' or
               m['marketName'] == 'Combo chance double & Total de buts'):
                for x in m['marketList']:
                    #if(m['marketName'] == 'Combo chance double & Total de buts'):
                        #print(m['marketList'][0]['selections'][0]['name'])
                    for j in x['selections']:
                        data_item = {"IdGame": idgame,
                                     "DateTime": DateTime,
                                     "Competition": competitionName,
                                     "Home": home,
                                     "Away": away,
                                     "Bet": j['name'],
                                     "Odd":round(1+float(j['currentPriceUp'])/float(j['currentPriceDown']),2)  
                                    }
                        row_data0.append(data_item)
            if(m['marketName'] == 'But pour les 2 équipes'):
                for j in m['marketList'][0]['selections']:
                    data_item = {"IdGame": idgame,
                                 "DateTime": DateTime,
                                 "Competition": competitionName,
                                 "Home": home,
                                 "Away": away,
                                 "Bet": j['name'],
                                 "Odd":round(1+float(j['currentPriceUp'])/float(j['currentPriceDown']),2)  
                                }
                    row_data0.append(data_item)
            if(m['marketName'] == 'Résultat du match' or
               m['marketName'] == 'Chance double' or
               m['marketName'] == 'Combo chance double & les 2 équipes marquent'):
                for j in m['marketList'][0]['selections']:
                    data_item = {"IdGame": idgame,
                                 "DateTime": DateTime,
                                 "Competition": competitionName,
                                 "Home": home,
                                 "Away": away,
                                 "Bet": j['name'],
                                 "Odd":round(1+float(j['currentPriceUp'])/float(j['currentPriceDown']),2)  
                                }
                    row_data0.append(data_item)

100%|██████████| 33/33 [00:06<00:00,  4.79it/s]


In [21]:
OddGames = pd.DataFrame(row_data0)
# next_sunday = datetime.date.today() + datetime.timedelta(days=(6 - datetime.date.today().weekday()) % 7 + 1)
# OddGames = OddGames[OddGames['DateTime'] < next_sunday.strftime('%Y-%m-%d 23:59:59')]
OddGames["DateTime"].unique()

Check = pd.DataFrame()
OddGames['Bet'] = OddGames['Bet'].apply(unidecode)
OddGames['Home'] = OddGames['Home'].apply(unidecode)
OddGames['Away'] = OddGames['Away'].apply(unidecode)
Check['BetOri'] = OddGames['Bet']
Check['IdGame'] = OddGames['IdGame']

OddGames["Bet"].replace(special_dict0, inplace=True, regex=True)
OddGames["Away"].replace(special_dict0, inplace=True, regex=True)
OddGames["Home"].replace(special_dict0, inplace=True, regex=True)

OddGames["Bet"].replace(special_dict, inplace=True, regex=True)
OddGames["Away"].replace(special_dict, inplace=True, regex=True)
OddGames["Home"].replace(special_dict, inplace=True, regex=True)
inverted_dict = {v: k for k, v in Dict_teams.items()}
OddGames["Away"].replace(inverted_dict, inplace=True)
OddGames["Home"].replace(inverted_dict, inplace=True)
OddGames["Bet"].replace(inverted_dict, inplace=True)
OddGames["Bet"] = OddGames.apply(lambda p: p["Bet"].replace(p["Away"], 'A') if (p["Away"] in p["Bet"]) else p["Bet"], axis=1)
OddGames["Bet"] = OddGames.apply(lambda p: p["Bet"].replace(p["Home"], 'H') if (p["Home"] in p["Bet"]) else p["Bet"], axis=1)

OddGames["Away"].replace(Dict_teams, inplace=True)
OddGames["Home"].replace(Dict_teams, inplace=True)
OddGames["Bet"].replace(Dict_teams, inplace=True)
OddGames["Bet"] = OddGames.apply(lambda p: p["Bet"].replace(p["Away"], 'A') if (p["Away"] in p["Bet"]) else p["Bet"], axis=1)
OddGames["Bet"] = OddGames.apply(lambda p: p["Bet"].replace(p["Home"], 'H') if (p["Home"] in p["Bet"]) else p["Bet"], axis=1)

OddGames["Bet"] = OddGames["Bet"].str.replace(r'Moins de ', 'M', regex=True).replace(r'Plus de ', 'P', regex=True).replace(r' ou ', '/', regex=True)
OddGames["Bet"] = OddGames["Bet"].str.replace(r'under ', 'M', regex=True).replace(r'over ', 'P', regex=True).replace(r' Or ', '/', regex=True).replace(r' or ', '/', regex=True)
OddGames["Bet"] = OddGames["Bet"].str.replace(r'Match nul', 'D', regex=True).replace(r'match nul', 'D', regex=True).replace(r'Match Nul', 'D', regex=True)
OddGames["Bet"] = OddGames["Bet"].str.replace(r'Draw', 'D', regex=True).replace(r'draw', 'D', regex=True).replace(r'Egalite', 'D', regex=True)
OddGames["Bet"] = OddGames["Bet"].replace(r'Oui', 'LDEM', regex=True).replace(r'Non', 'LDEMP', regex=True).replace(r' ', '', regex=True)
OddGames["Bet"] = OddGames["Bet"].replace(r'Yes', 'LDEM', regex=True).replace(r'No', 'LDEMP', regex=True).replace(r' ', '', regex=True)
OddGames['Bet'] = OddGames['Bet'].str.replace('.', ',', regex=False)

#OddGames.head(7)
my_list = list(set(OddGames["Bet"]))
#OddGames = OddGames.reset_index(level=1)
#OddGames = OddGames.pivot_table(values='Bet', index=OddGames.index, columns='IdGame')
OddGames2 = OddGames.set_index(['IdGame', 'DateTime', 'Home', 'Away', 'Competition']).pivot(columns='Bet').reindex()#.rename_axis(None, axis=1)

tickers_data = []
for ticker, data in OddGames2.groupby(level=0, axis=1):
    data.columns = data.columns.droplevel(0)
    data = data.reset_index()
    data.insert(0, 'ticker', ticker)
    tickers_data.append(data)

OddGames2 = pd.concat(tickers_data).drop(['ticker'], axis=1).rename_axis(None, axis=1)
#OddGames2.columns
#OddGames2.head(7)


cols = OddGames2.columns.values
Cols = [
'IdGame', 'DateTime', 'Home', 'Away', 'Competition', 
'A', 'D', 'H',  'H/D', 'D/A', 'H/A', 
    
'H/D&M1,5', 'H/D&M2,5', 'H/D&M3,5', 'H/D&M4,5',
'H/D&P1,5', 'H/D&P2,5', 'H/D&P3,5', 'H/D&P4,5', 
    
'D/A&M1,5', 'D/A&M2,5', 'D/A&M3,5', 'D/A&M4,5',
'D/A&P1,5', 'D/A&P2,5', 'D/A&P3,5', 'D/A&P4,5', 
    
'H/A&M1,5', 'H/A&M2,5', 'H/A&M3,5', 'H/A&M4,5',
'H/A&P1,5', 'H/A&P2,5', 'H/A&P3,5', 'H/A&P4,5', 
    
'LDEM', 'LDEMP', 
'H/D&LDEM', 'H/D&LDEMP', 
'D/A&LDEM', 'D/A&LDEMP', 
'H/A&LDEM', 'H/A&LDEMP', 
    
'M0,5', 'M1,5', 'M2,5', 'M3,5', 'M4,5', 'M5,5',
'P0,5', 'P1,5', 'P2,5', 'P3,5', 'P4,5', 'P5,5']
    
OddGames3 = OddGames2[Cols].reset_index(drop=True)
OddGames3['DateTime'] = pd.to_datetime(OddGames3['DateTime'])

WrongCol = []
for col in cols:
    if col not in OddGames3.columns.values:
        WrongCol.append(col)
        
mask = ~OddGames['Bet'].isin(Cols)
bugCols2 = OddGames[mask]['Bet'].tolist()
Teams = [OddGames['Away'].unique(), OddGames['Home'].unique()]
bugCols = Check[mask]['BetOri'].tolist()
# print(len(bugCols2))
print(Check[mask]['IdGame'].tolist())
# print(len(bugCols))
print(bugCols2)

Unibet = OddGames3.sort_values(by='DateTime')

['2676305_1', '2676305_1', '2676305_1', '2676305_1', '2676305_1', '2676305_1', '2676305_1', '2676305_1', '2676305_1', '2676305_1', '2676305_1', '2646792_1', '2646792_1', '2646792_1', '2646792_1', '2646792_1', '2646792_1', '2646792_1', '2646792_1', '2646792_1', '2646792_1']
['H/1A', 'H/1A&M1,5', 'H/1A&P1,5', 'H/1A&M2,5', 'H/1A&P2,5', 'H/1A&M3,5', 'H/1A&P3,5', 'H/1A&M4,5', 'H/1A&P4,5', 'H/1A&LDEM', 'H/1A&LDEMP', 'H/1FCA&M1,5', 'H/1FCA&P1,5', 'H/1FCA&M2,5', 'H/1FCA&P2,5', 'H/1FCA&M3,5', 'H/1FCA&P3,5', 'H/1FCA&M4,5', 'H/1FCA&P4,5', 'H/1FCA&LDEM', 'H/1FCA&LDEMP']


In [22]:
# Save UNIBET odds
if True:
# if Unibet.isna().mean()[Unibet.isna().mean() != 0].mean() < 0.01:
    #Save odds as CSV
    csv = pd.read_csv("Odds_Match.csv")
    def bool(p):
        if any(csv['IdGame'].str.contains(str(p))):
            return(False)
        else:
            return(True)

    new = Unibet.apply(lambda p : bool(p["IdGame"]), axis=1)
    csv = pd.concat([csv,Unibet[new]])
    csv['DateTime'] = pd.to_datetime(csv['DateTime'])
    csv.sort_values(by='DateTime').to_csv('Odds_Match.csv', index=False, encoding='utf-8')